In [39]:
import boto3
import pandas as pd
import numpy as np
from io import StringIO
'''
# Define your bucket name and file key (file path in S3)
BUCKET_NAME = "blue-blood-data"
FILE_KEY = "BigQuery Blue Blood DB Data.csv"  # Change to your actual file path in S3

# Create an S3 client
s3 = boto3.client("s3")

# Fetch the file from S3
response = s3.get_object(Bucket=BUCKET_NAME, Key=FILE_KEY)

# Read the CSV file into a pandas DataFrame
csv_content = response["Body"].read().decode("utf-8")
df = pd.read_csv(StringIO(csv_content))
'''
df = pd.read_csv(r"C:\Users\advay\CertificationProjects\Blue-Blood-ACM-Research\sample-final-data.csv")

# Print DataFrame
print(df)

    subject_id   prescription_start               prescription_drug  \
0        10013  2125-10-05T00:00:00               Phenylephrine HCl   
1        10013  2125-10-05T00:00:00                     Vasopressin   
2        10013  2125-10-05T00:00:00            Iso-Osmotic Dextrose   
3        10013  2125-10-05T00:00:00                      Dobutamine   
4        10013  2125-10-05T00:00:00                         Aspirin   
5        10013  2125-10-05T00:00:00               Magnesium Sulfate   
6        10013  2125-10-05T00:00:00                Morphine Sulfate   
7        10013  2125-10-05T00:00:00             Pantoprazole Sodium   
8        10013  2125-10-05T00:00:00                      Furosemide   
9        10013  2125-10-05T00:00:00                    Azithromycin   
10       10013  2125-10-05T00:00:00                              NS   
11       10013  2125-10-05T00:00:00                   Acetaminophen   
12       10013  2125-10-05T00:00:00                             D5W   
13    

In [40]:
cols = df.columns
print(cols)

df.head()

Index(['subject_id', 'prescription_start', 'prescription_drug',
       'prescription_dose_val_rx', 'prescription_dose_unit_rx',
       'pre_charttime', 'pre_ph', 'pre_pco2', 'pre_po2', 'pre_bicarbonate',
       'pre_baseexcess', 'pre_totalco2', 'pre_hematocrit', 'pre_hemoglobin',
       'pre_sodium', 'pre_potassium', 'pre_chloride', 'pre_glucose',
       'pre_lactate', 'pre_so2', 'pre_spo2', 'pre_fio2_chartevents',
       'pre_aado2_calc', 'pre_pao2fio2', 'pre_temperature', 'pre_fio2',
       'pre_aado2', 'pre_carboxyhemoglobin', 'pre_methemoglobin',
       'pre_calcium', 'pre_intubated', 'post_charttime', 'post_ph',
       'post_pco2', 'post_po2', 'post_bicarbonate', 'post_baseexcess',
       'post_totalco2', 'post_hematocrit', 'post_hemoglobin', 'post_sodium',
       'post_potassium', 'post_chloride', 'post_glucose', 'post_lactate',
       'post_so2', 'post_spo2', 'post_fio2_chartevents', 'post_aado2_calc',
       'post_pao2fio2', 'post_temperature', 'post_fio2', 'post_aado2',
      

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,60,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10013,2125-10-05T00:00:00,Vasopressin,100,UNIT,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,50,ml,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10013,2125-10-05T00:00:00,Dobutamine,250,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10013,2125-10-05T00:00:00,Aspirin,325,mg,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [41]:
#PREPROCESSING --> normalizing unit labels + one hot encoding

units = df['prescription_dose_unit_rx'].to_list()

#Normalize the units
units = [unit.lower() for unit in units]
print("All units in sample DF:")
print(units)

set_of_units = set()
for unit in units:
    if unit not in set_of_units:
        set_of_units.add(unit)
print("Set of UNITS: ")
print(set_of_units)

#Mapping for One Hot Encoding
label_encoding = {}
mapping = 0
for elem in set_of_units:
    label_encoding[elem] = mapping
    mapping += 1
print(label_encoding)

All units in sample DF:
['mg', 'unit', 'ml', 'mg', 'mg', 'gm', 'mg', 'mg', 'mg', 'mg', 'ml', 'mg', 'ml', 'neb', 'mg', 'mg', 'unit', 'gm', 'mg', 'mg', 'ml', 'mg', 'ml', 'ml', 'gm', 'mg', 'ml', 'ml', 'mg', 'meq', 'mg', 'mg', 'meq', 'mg', 'mg', 'appl', 'ml', 'meq', 'ml', 'meq', 'ml', 'mg', 'mg', 'unit', 'mg', 'gm', 'unit', 'mg', 'mg', 'meq']
Set of UNITS: 
{'ml', 'unit', 'neb', 'gm', 'mg', 'appl', 'meq'}
{'ml': 0, 'unit': 1, 'neb': 2, 'gm': 3, 'mg': 4, 'appl': 5, 'meq': 6}


In [42]:
df['prescription_dose_unit_rx'] = df['prescription_dose_unit_rx'].str.lower().map(label_encoding)

In [43]:
df.head(10)

,subject_id,prescription_start,prescription_drug,prescription_dose_val_rx,prescription_dose_unit_rx,pre_charttime,pre_ph,pre_pco2,pre_po2,pre_bicarbonate,...,post_fio2_chartevents,post_aado2_calc,post_pao2fio2,post_temperature,post_fio2,post_aado2,post_carboxyhemoglobin,post_methemoglobin,post_calcium,post_intubated
0,10013,2125-10-05T00:00:00,Phenylephrine HCl,60,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,10013,2125-10-05T00:00:00,Vasopressin,100,1,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,10013,2125-10-05T00:00:00,Iso-Osmotic Dextrose,50,0,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10013,2125-10-05T00:00:00,Dobutamine,250,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,10013,2125-10-05T00:00:00,Aspirin,325,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,10013,2125-10-05T00:00:00,Magnesium Sulfate,2,3,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,10013,2125-10-05T00:00:00,Morphine Sulfate,100,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,10013,2125-10-05T00:00:00,Pantoprazole Sodium,40,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,10013,2125-10-05T00:00:00,Furosemide,20,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,10013,2125-10-05T00:00:00,Azithromycin,250,4,2125-10-04T23:59:00,7.3,63.0,60.0,NaN,...,60.000002,231.550017,191.666659,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [55]:
#BERT FOR VECTOR EMBEDDINGS!
import tensorflow_hub as hub
import tensorflow_text as text

preprocess_url = "https://kaggle.com/models/tensorflow/bert/TensorFlow2/en-uncased-preprocess/3"
encoder_url = "https://www.kaggle.com/models/tensorflow/bert/TensorFlow2/bert-en-uncased-l-10-h-128-a-2/2"

#BERT Preprocessor
bert_preprocess_model = hub.KerasLayer(preprocess_url)
# Encoder layer
bert_encoder = hub.KerasLayer(encoder_url, trainable=True)

In [67]:
# Iterate through the prescription_drug col
rx_vector_embeddings = []

for individual_rx in df['prescription_drug']:
    # print(type(individual_rx))
    preprocessed_rx_text = bert_preprocess_model([str(individual_rx)])

    #Acts like a function, pass preprocessed input as a param
    bert_results = bert_encoder(preprocessed_rx_text)

    #Reverse the process above --> embeddings are as a list of tensors
    #get last encoder layer's tensor --> most abstract
    encoder_output = bert_results["encoder_outputs"][-1]

    print(f"Shape of raw encoder output for {individual_rx}: {encoder_output.shape}")
    break


Shape of raw encoder output for Phenylephrine HCl: (1, 128, 128)
